# Web Scrapping 

## Author: Grupo 4 
## Integrantes: Lisbeth Ccoyo, Mishell Delgado, Steven Atoche

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore') # eliminar warning messages 

from selenium import webdriver  # manipulación de driver 
from webdriver_manager.chrome import ChromeDriverManager # manejar diferentes versiones del driver

import re # expresiones regulares 
import time # time 
from selenium.webdriver.support.ui import Select  # Trabaja con el tag <select></select>
import os
import sys
from selenium.webdriver.common.by import By  # permite seleccionar los elementos en un html
warnings.filterwarnings('ignore') # eliminar warning messages 

from selenium.webdriver.common.keys import Keys  # ingresar información a la página web (nombres, fechas)
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains # movilizarnos en la página web 
from tqdm import tqdm

## 1. Limpiar la base de datos 

In [2]:
# Subiendo la base de datos "resultado_2014"

data1 = pd.read_stata("../../data/trabajo_final/CE/resultado_2014.dta")
data = data1.copy()

,COD_MOD,ANEXO,TIPOREG,NROCED,CUADRO,TIPDATO,DESCRIP,D01,D02,D03,...,D19,D20,FORMAS,NIV_MOD,TIPOPROG,GES_DEP,CODOOII,CODGEO,AREA_MED,IMPUTADO
0,0001506,0,9,02,101,01,Concluyeron,0,0,0,...,0,0,N,A5,4,A1,150206,151011,2,0
1,0001506,0,9,02,101,03,Trasladados a otro P.E./I.E.,0,0,0,...,0,0,N,A5,4,A1,150206,151011,2,0
2,0001507,0,9,02,101,01,Concluyeron,0,0,0,...,0,0,N,A5,4,A1,150206,151003,2,0
3,0001509,0,9,02,101,01,Concluyeron,0,0,0,...,0,0,N,A5,4,A1,150206,151001,2,0
4,0001512,0,9,02,101,01,Concluyeron,0,0,0,...,0,0,N,A5,4,A1,150206,151003,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308871,1501212,0,1,08,101,02,Retirados,0,0,0,...,0,0,N,E0,a,A1,150106,150132,1,1
308872,1501212,0,1,08,104,01,Situación económica,0,0,0,...,0,0,N,E0,a,A1,150106,150132,1,1
308873,0901835,0,1,08,101,01,Concluyeron,2,2,5,...,0,0,N,E0,a,A1,150106,150132,1,1
308874,0901835,0,1,08,101,02,Retirados,0,0,0,...,0,0,N,E0,a,A1,150106,150132,1,1


In [3]:
# Observamos la base de datos
## tiene códigos repetidos
data

,COD_MOD,ANEXO,TIPOREG,NROCED,CUADRO,TIPDATO,DESCRIP,D01,D02,D03,...,D19,D20,FORMAS,NIV_MOD,TIPOPROG,GES_DEP,CODOOII,CODGEO,AREA_MED,IMPUTADO
0,0001506,0,9,02,101,01,Concluyeron,0,0,0,...,0,0,N,A5,4,A1,150206,151011,2,0
1,0001506,0,9,02,101,03,Trasladados a otro P.E./I.E.,0,0,0,...,0,0,N,A5,4,A1,150206,151011,2,0
2,0001507,0,9,02,101,01,Concluyeron,0,0,0,...,0,0,N,A5,4,A1,150206,151003,2,0
3,0001509,0,9,02,101,01,Concluyeron,0,0,0,...,0,0,N,A5,4,A1,150206,151001,2,0
4,0001512,0,9,02,101,01,Concluyeron,0,0,0,...,0,0,N,A5,4,A1,150206,151003,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308871,1501212,0,1,08,101,02,Retirados,0,0,0,...,0,0,N,E0,a,A1,150106,150132,1,1
308872,1501212,0,1,08,104,01,Situación económica,0,0,0,...,0,0,N,E0,a,A1,150106,150132,1,1
308873,0901835,0,1,08,101,01,Concluyeron,2,2,5,...,0,0,N,E0,a,A1,150106,150132,1,1
308874,0901835,0,1,08,101,02,Retirados,0,0,0,...,0,0,N,E0,a,A1,150106,150132,1,1


In [4]:
# Eliminando los datos duplicados

data = data.drop_duplicates(subset='COD_MOD', keep='first')

In [5]:
data

,COD_MOD,ANEXO,TIPOREG,NROCED,CUADRO,TIPDATO,DESCRIP,D01,D02,D03,...,D19,D20,FORMAS,NIV_MOD,TIPOPROG,GES_DEP,CODOOII,CODGEO,AREA_MED,IMPUTADO
0,0001506,0,9,02,101,01,Concluyeron,0,0,0,...,0,0,N,A5,4,A1,150206,151011,2,0
2,0001507,0,9,02,101,01,Concluyeron,0,0,0,...,0,0,N,A5,4,A1,150206,151003,2,0
3,0001509,0,9,02,101,01,Concluyeron,0,0,0,...,0,0,N,A5,4,A1,150206,151001,2,0
4,0001512,0,9,02,101,01,Concluyeron,0,0,0,...,0,0,N,A5,4,A1,150206,151003,2,0
5,0001513,0,9,02,101,01,Concluyeron,0,0,0,...,0,0,N,A5,4,A1,150206,151003,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308861,1197151,0,1,08,101,01,Concluyeron,3,2,9,...,0,0,N,E0,a,A1,150102,150119,1,1
308864,0901777,0,1,08,101,01,Concluyeron,1,1,8,...,0,0,N,E0,a,A1,150106,150132,1,1
308867,1230994,0,1,08,101,01,Concluyeron,5,4,13,...,0,0,N,E0,a,A1,150106,150111,1,1
308870,1501212,0,1,08,101,01,Concluyeron,1,1,3,...,0,0,N,E0,a,A1,150106,150132,1,1


## Extraer los datos de latitud y longitud de  los centros educativos y unirlo a la única base de datos

In [10]:
# Iniciando el navegador web"
driver= webdriver.Chrome( "chromedriver.exe" )

# Maximize window
driver.maximize_window()
driver.get('https://sigmed.minedu.gob.pe/mapaeducativo/')

In [11]:
# Click en Aceptar
driver.find_element(By.XPATH,'//*[@id="winuseterms"]/div/div[3]/button').click()

In [12]:
# Iterar sobre los códigos en la columna COD_MOD

for codigo in data['COD_MOD']:
    # Encontrar el campo de entrada de texto y escribir el código
    keyword = driver.find_element(By.ID, "txttextsearch")
    keyword.clear()
    keyword.send_keys(codigo)
    
    print(codigo)
    
    # Hacer clic en el botón de búsqueda
    driver.find_element(By.XPATH,'//*[@id="searchbox"]/table/tbody/tr[1]/td/form/table/tbody/tr[7]/td/div/button[1]').click()
    
    # Esperar a que aparezca la tabla de resultados
    driver.implicitly_wait(4)  # Esperar hasta 10 segundos
    
     # Extraer la información de latitud y longitud de la tabla de resultados
    time.sleep(2)  # Esperar 2 segundos antes de extraer la información
  
    try:
        latitud_element = driver.find_element(By.XPATH, '//*[@id="gvwDataPadron"]/tbody/tr[2]/td[14]')
        longitud_element = driver.find_element(By.XPATH, '//*[@id="gvwDataPadron"]/tbody/tr[2]/td[15]')
        latitud = latitud_element.text
        longitud = longitud_element.text
    except NoSuchElementException:
        latitud = '.'
        longitud = '.'
    
    # Actualizar la base de datos resultado_2014 con las columnas de latitud y longitud
    data.loc[data['COD_MOD'] == codigo, 'Latitud'] = latitud
    data.loc[data['COD_MOD'] == codigo, 'Longitud'] = longitud

 

0001506
0001507
0001509
0001512
0001513
0001514
0001516
0002212
0002317
0002319
0002320
0002323
0002325
0002327
0002328
0002330
0002331
0002332
0003124
0003125
0003128
0003130
0003131
0003132
0003133
0003135
0003138
0003832
0003834
0003836
0003837
0003838
0003843
0004602
0004603
0004605
0004607
0004609
0004610
0004611
0004612
0004613
0004616
0004617
0005602
0005610
0005638
0005647
0005650
0005651
0005654
0005655
0005656
0005657
0005659
0005661
0005665
0005666
0005667
0005668
0005669
0005670
0005671
0005672
0005673
0008022
0008025
0010613
0010626
0010628
0010632
0010633
0010634
0010635
0010636
0013002
0013004
0013006
0013009
0013010
0013018
0013019
0013021
0022102
0022108
0022109
0045902
0045903
0045904
0045909
0045911
0045912
0045917
0049124
0049129
0049808
0049814
0049815
0050201
0050205
0050206
0050207
0050902
0050903
0050905
0050911
0050912
0050913
0050916
0051001
0051703
0051708
0051709
0051711
0051713
0051716
0051717
0051723
0051724
0051732
0051733
0051734
0051735
0051736
0053304


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00356E73+48323]
	(No symbol) [0x002E9661]
	(No symbol) [0x001F5308]
	(No symbol) [0x001E7D46]
	(No symbol) [0x001E7A69]
	(No symbol) [0x001F6770]
	(No symbol) [0x0024C678]
	(No symbol) [0x0023A6D6]
	(No symbol) [0x0021847C]
	(No symbol) [0x0021957D]
	GetHandleVerifier [0x005BFD5D+2575277]
	GetHandleVerifier [0x005FF86E+2836158]
	GetHandleVerifier [0x005F96DC+2811180]
	GetHandleVerifier [0x003E41B0+626688]
	(No symbol) [0x002F314C]
	(No symbol) [0x002EF4B8]
	(No symbol) [0x002EF59B]
	(No symbol) [0x002E21B7]
	BaseThreadInitThunk [0x76287D59+25]
	RtlInitializeExceptionChain [0x7717B74B+107]
	RtlClearBits [0x7717B6CF+191]


In [15]:
# Resultados

data_5_rows

,COD_MOD,ANEXO,TIPOREG,NROCED,CUADRO,TIPDATO,DESCRIP,D01,D02,D03,...,FORMAS,NIV_MOD,TIPOPROG,GES_DEP,CODOOII,CODGEO,AREA_MED,IMPUTADO,Latitud,Longitud
0,0001506,0,9,02,101,01,Concluyeron,0,0,0,...,N,A5,4,A1,150206,151011,2,0,-12.63774000,-75.53330900
2,0001507,0,9,02,101,01,Concluyeron,0,0,0,...,N,A5,4,A1,150206,151003,2,0,-12.63094300,-75.97891900
3,0001509,0,9,02,101,01,Concluyeron,0,0,0,...,N,A5,4,A1,150206,151001,2,0,-12.58617700,-75.95000000
4,0001512,0,9,02,101,01,Concluyeron,0,0,0,...,N,A5,4,A1,150206,151003,2,0,-12.62320200,-75.98711300
5,0001513,0,9,02,101,01,Concluyeron,0,0,0,...,N,A5,4,A1,150206,151003,2,0,-12.65254200,-75.97808700
6,0001514,0,9,02,101,01,Concluyeron,0,0,0,...,N,A5,4,A1,150206,151003,2,0,-12.70799500,-75.97021000
7,0001516,0,9,02,101,01,Concluyeron,0,0,0,...,N,A5,4,A1,150206,151011,2,0,.,.
8,0002212,0,1,03,101,01,Aprobados,8,15,6,...,S,F0,a,A1,200001,200304,2,0,-5.88915535,-79.53830349
18,0002317,0,9,02,101,01,Concluyeron,0,0,0,...,N,A5,4,A1,150206,151008,2,0,-12.71781800,-75.91456700
19,0002319,0,9,02,101,01,Concluyeron,0,0,0,...,N,A5,4,A1,150206,151009,2,0,-12.85206300,-76.00715600


# GRACIAS